<a href="https://colab.research.google.com/github/VictoorV/movie_classif_lstm/blob/main/Film_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.

In [2]:
pip install portalocker

In [3]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import IMDB
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence


In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = get_tokenizer('basic_english')

In [64]:
train_iter, test_iter = IMDB()

for label, text in train_iter:
    print(f"Label: {label}, Texte: {text[:200]}...\n")
    break

Label: 1, Texte: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev...



In [65]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocabulary = build_vocab_from_iterator(
    yield_tokens(IMDB()[0]),
    specials=["<pad>", "<unk>"])
vocabulary.set_default_index(vocabulary["<unk>"])

In [66]:
print(len(vocabulary))
print(vocabulary['<pad>'])
print(vocabulary["."])
print(vocabulary.get_itos()[0])

100684
0
3
<pad>


In [19]:
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
print(f'Taille du train_dataset : {len(train_dataset)}')
print(f'Taille du test_dataset : {len(test_dataset)}')

Taille du train_dataset : 25000
Taille du test_dataset : 25000


In [20]:
sequences = [
    (2, "This movie was really great !"),
    (1, "I am not sure about the scenario."),
    (1, "It was not as good as I expected."),
    (2, "The actors were good.")
]

In [244]:
sort = True

def collate_batch(batch, sort = sort):
  labels, samples = zip(*batch)
  labels = torch.tensor(labels, dtype=torch.int64) - 1
  text_lengths = torch.tensor([len(tokenizer(sample)) for sample in samples], dtype=torch.int64)
  inputs = [torch.tensor(vocabulary(tokenizer(sample)), dtype=torch.int64) for sample in samples]
  inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
  if sort:
    text_lengths, indices = torch.sort(text_lengths, descending=True)
    labels = labels[indices]
    inputs = inputs[indices]
  return labels, inputs, text_lengths

In [266]:
labels, inputs, text_lengths = collate_batch(sequences, sort)

In [248]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [256]:
class LSTMModel(torch.nn.Module):
    def __init__(self,
                 vocab_size,
                 embedding_size,
                 hidden_size,
                 num_classes):
        super().__init__()

        # Embedding field
        self.embedding = torch.nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_size)

        # LSTM cell
        self.rnn = torch.nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            batch_first=True)

        # Fully connected output
        self.fc = torch.nn.Linear(
            hidden_size, num_classes)

    def forward(self, text_tokens, text_lengths):
        # Extract embedding vectors
        embeddings = self.embedding(
            text_tokens)
        packed = pack_padded_sequence(embeddings, text_lengths, batch_first=True, enforce_sorted=sort)
        h_t, (h_n, c_n) = self.rnn(packed)

        return self.fc(h_n[-1])

In [257]:
def train_model(model, cost_function, optimizer, data_loader):
    model.to(device)

    model.train()

    current_loss = 0.0
    current_acc = 0.0

    for i, (labels, inputs, text_lengths) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs, text_lengths)
        _, predictions = torch.max(outputs, 1)
        loss = cost_function(outputs, labels)

        loss.backward()
        optimizer.step()

        current_loss += loss.item() * labels.size(0)
        current_acc += torch.sum(predictions == labels.data)

    total_loss = current_loss / len(data_loader.dataset)
    total_acc = current_acc.double() / len(data_loader.dataset)

    print('Train Loss: {:.4f}; Accuracy: {:.4f}'.format(total_loss, total_acc))

def test_model(model, cost_function, data_loader):
    model.to(device)

    model.eval()

    current_loss = 0.0
    current_acc = 0.0

    for i, (labels, inputs, text_lengths) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs, text_lengths)
            _, predictions = torch.max(outputs, 1)
            loss = cost_function(outputs, labels)

        current_loss += loss.item() * labels.size(0)
        current_acc += torch.sum(predictions == labels.data)

    total_loss = current_loss / len(data_loader.dataset)
    total_acc = current_acc.double() / len(data_loader.dataset)

    print('Test Loss: {:.4f}; Accuracy: {:.4f}'.format(total_loss, total_acc))

In [263]:
model = LSTMModel(
    vocab_size=len(vocabulary),
    embedding_size=64,
    hidden_size=64,
    num_classes=2)

model.to(device)

cost_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [264]:
for epoch in range(5):
    print(f'Epoch: {epoch + 1}')
    train_model(model, cost_fn, optim, train_dataloader)
    test_model(model, cost_fn, test_dataloader)

Epoch: 1
Train Loss: 0.6556; Accuracy: 0.6105
Test Loss: 0.6659; Accuracy: 0.5899
Epoch: 2
Train Loss: 0.5754; Accuracy: 0.6961
Test Loss: 0.5040; Accuracy: 0.7575
Epoch: 3
Train Loss: 0.4441; Accuracy: 0.8017
Test Loss: 0.4409; Accuracy: 0.8058
Epoch: 4
Train Loss: 0.3555; Accuracy: 0.8498
Test Loss: 0.4293; Accuracy: 0.8164
Epoch: 5
Train Loss: 0.3018; Accuracy: 0.8769
Test Loss: 0.3840; Accuracy: 0.8386
